In [1]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image, ImageDraw, ImageFont
import requests
from io import BytesIO

c:\Users\FerielKadri\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class DocumentTableDetector:
    def __init__(self):
        """Initialize the processor and model."""
        self.processor = DetrImageProcessor.from_pretrained("TahaDouaji/detr-doc-table-detection")
        self.model = DetrForObjectDetection.from_pretrained("TahaDouaji/detr-doc-table-detection")
        self.model.eval()  # Set the model to evaluation mode
        
    def load_image(self, image_path):
        """Helper function to load an image from a URL or local path."""
        try:
            if image_path.startswith('http'):
                response = requests.get(image_path)
                image = Image.open(BytesIO(response.content))
            else:
                image = Image.open(image_path)
            return image.convert("RGB")
        except Exception as e:
            raise ValueError(f"Error loading image: {str(e)}")
            
    def predict(self, image_path):
        """Predict tables in an image."""
        try:
                image = self.load_image(image_path)
                inputs = self.processor(images=image, return_tensors="pt")
                outputs = self.model(**inputs)

                # Converting outputs to COCO API and filtering by score threshold
                target_sizes = torch.tensor([image.size[::-1]])
                results = self.processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

                predictions = []
                for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
                    box = [round(i, 2) for i in box.tolist()]
                    predictions.append(
                        {
                            "label": self.model.config.id2label[label.item()],
                            "confidence": round(score.item(), 3),
                            "box": box
                        }
                    )
                return predictions
        except Exception as e:
                raise ValueError(f"Error processing image: {str(e)}")
                
    def draw_boxes(self, predictions, image_path, show_scores=False):
        """
        Draw bounding boxes around detected tables on the image.
        Optionally, include detection confidence scores above the boxes.

        Args:
            predictions (list of dicts): List of predictions with scores, labels, and boxes.
            image_path (str): Path to the image file.
            show_scores (bool): If True, display the confidence scores above the bounding boxes.

        Returns:
            PIL.Image: The image with drawn bounding boxes.
        """
        image = self.load_image(image_path)
        draw = ImageDraw.Draw(image)

        # Use a larger font for drawing text, increase the font size 
        try:
            font = ImageFont.truetype("arial.ttf", 70) 
        except IOError:
            # Use a default font with larger size if arial.ttf is not available
            font = ImageFont.load_default(size=20)

        for prediction in predictions:
            box = prediction["box"]
            score = prediction["confidence"]
            # Draw the bounding box
            draw.rectangle(box, outline="red", width=2)

            # Optionally display the confidence score above the box
            if show_scores:
                score_text = f"{score:.2f}"
                # Position the text above the box
                text_position = (box[0], box[1] - 60)  # Adjust position slightly higher for larger text
                draw.text(text_position, score_text, fill="blue", font=font)

        return image

In [3]:
detector = DocumentTableDetector()
image_path = 'invoice.jpg'
predictions = detector.predict(image_path) 

try:
    image_with_boxes = detector.draw_boxes(predictions, image_path, show_scores=True)
    image_with_boxes.show()  
except ValueError as error:
    print("An error occurred:", error)


The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
Some weights of the model checkpoint at TahaDouaji/detr-doc-table-detection were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification mode